# Oxford Man Institute NLP Tutorial 

## 1. Introduction 

There are several ways of performing sentiment classification on a document or article, ranging from word-counts to modern Transformer-based Language Models. In this tutorial we will take you through a range of classification techniques:
- Loughran & McDonald financial sentiment dictionary
- Naive Bayes Classifier
- BERT out of the box
- BERT fine-tuned on general sentiment datasets
- FinBERT 
    - BERT that has been trained on positive and negative financial documents

## 2. Traditional sentiment analysis

### Import packages and load dictionaries

In [1]:
import numpy as np
import re

### Loughran & McDonald classifier

Loughran & McDonald released their master dictionary in 2011 in conjunction with their paper “When is a Liability not a Liability? Textual Analysis, Dictionaries, and 10-Ks". The dictionary lists a number of words and includes negative, positive, uncertainty, litigious, strong modal, weak modal, and constraining tags. 

There are several shortcomings to this simplistic approach:

- **Some words don't appear in the dictionary (fall, rise, etc.)**
- **Some words are negative/positive given the context they are written (profit, expenditure, etc.)**
- **Simple counts of words don't necessarily infer the overall sentiment**
    - *Hatred for football has always confused me; there are so many haters who attack the sport, but I have always loved it.* - 3 negative words and 1 positive word.


We have taken the words that have a negative and positive tag for our classifier:

In [2]:
lmdict = np.load('data/LoughranMcDonald_dict.npy', allow_pickle='TRUE').item()
print('Some examples of negative words: ', lmdict['Negative'][:5])
print('Some examples of positive words: ', lmdict['Positive'][:5])

Some examples of negative words:  ['abandon', 'abandoned', 'abandoning', 'abandonment', 'abandonments']
Some examples of positive words:  ['able', 'abundance', 'abundant', 'acclaimed', 'accomplish']


In [3]:
lmdict['Positive'][:5]

['able', 'abundance', 'abundant', 'acclaimed', 'accomplish']

Check to see if a word appears in the dictionary:

In [4]:
word = 'fall'

if word in lmdict['Negative']:
    print(f'Yes, {word} is a Negative word in the Loughran & McDonald dictionary')
elif word in lmdict['Positive']:
    print(f'Yes, {word} is Positive word in the Loughran & McDonald dictionary')
else:
    print(f'No, {word} is not in the Loughran & McDonald dictionary')

No, fall is not in the Loughran & McDonald dictionary


Negation is another challenge that emerges using this approach. A techy fix is to check if the word is preceeded by a negating word in our list:

In [5]:
negate = ["aint", "arent", "cannot", "cant", "couldnt", "darent", "didnt", "doesnt", "ain't", "aren't", "can't",
          "couldn't", "daren't", "didn't", "doesn't", "dont", "hadnt", "hasnt", "havent", "isnt", "mightnt", "mustnt",
          "neither", "don't", "hadn't", "hasn't", "haven't", "isn't", "mightn't", "mustn't", "neednt", "needn't",
          "never", "none", "nope", "nor", "not", "nothing", "nowhere", "oughtnt", "shant", "shouldnt", "wasnt",
          "werent", "oughtn't", "shan't", "shouldn't", "wasn't", "weren't", "without", "wont", "wouldnt", "won't",
          "wouldn't", "rarely", "seldom", "despite", "no", "nobody"]

In [6]:
def negated(word):
    """
    Determine if preceding word is a negation word
    """
    if word.lower() in negate:
        return True
    else:
        return False

This function counts the number of negative and positive words in a document and performs a negation check to switch the polarity of words that are preceeded by a word in the *negate* list.

In [7]:
def tone_count_with_negation_check(dict, article):
    """
    Count positive and negative words with negation check. Account for simple negation only for positive words.
    Simple negation is taken to be observations of one of negate words occurring within three words
    preceding a positive words.
    """
    pos_count = 0
    neg_count = 0
 
    pos_words = []
    neg_words = []
 
    input_words = re.findall(r'\b([a-zA-Z]+n\'t|[a-zA-Z]+\'s|[a-zA-Z]+)\b', article.lower())
 
    word_count = len(input_words)
 
    for i in range(0, word_count):
        if input_words[i] in dict['Negative']:
            neg_count += 1
            neg_words.append(input_words[i])
        if input_words[i] in dict['Positive']:
            if i >= 3:
                if negated(input_words[i - 1]) or negated(input_words[i - 2]) or negated(input_words[i - 3]):
                    neg_count += 1
                    neg_words.append(input_words[i] + ' (with negation)')
                else:
                    pos_count += 1
                    pos_words.append(input_words[i])
            elif i == 2:
                if negated(input_words[i - 1]) or negated(input_words[i - 2]):
                    neg_count += 1
                    neg_words.append(input_words[i] + ' (with negation)')
                else:
                    pos_count += 1
                    pos_words.append(input_words[i])
            elif i == 1:
                if negated(input_words[i - 1]):
                    neg_count += 1
                    neg_words.append(input_words[i] + ' (with negation)')
                else:
                    pos_count += 1
                    pos_words.append(input_words[i])
            elif i == 0:
                pos_count += 1
                pos_words.append(input_words[i])
 
    print('The results with negation check:', end='\n\n')
    print('The # of positive words:', pos_count)
    print('The # of negative words:', neg_count)
    print('The list of found positive words:', pos_words)
    print('The list of found negative words:', neg_words)
    print('\n', end='')
 
    results = [word_count, pos_count, neg_count, pos_words, neg_words]
 
    return results
 
    
# A sample output
article = '''Pharmaceuticals group Orion Corp reported a fall in its third-quarter earnings that were hit by larger expenditures on R&D and marketing not able abandon'''
 
tone_count_with_negation_check(lmdict, article)

The results with negation check:

The # of positive words: 0
The # of negative words: 2
The list of found positive words: []
The list of found negative words: ['able (with negation)', 'abandon']



[26, 0, 2, [], ['able (with negation)', 'abandon']]

## 3. BERT classification

In [8]:
# Import all dependencies 
from datasets import load_dataset
from tqdm import tqdm
import pandas as pd
from transformers import AutoTokenizer,logging
from transformers import AutoModelForSequenceClassification, AutoTokenizer,BertForMaskedLM,AutoTokenizer
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from transformers import  TrainingArguments, Trainer, EarlyStoppingCallback
from transformers import AutoConfig
import warnings
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings("ignore")

C:\Users\drago\anaconda3\envs\myenv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
logging.set_verbosity_error()

In [10]:
# Import the dataset from huggingfaces' dataset repository
#fin_dataset = load_dataset('financial_phrasebank', 'sentences_allagree')
#df = pd.DataFrame(fin_dataset['train']) # send  it to a pandas dataframe

# If you are having issues running the code above on Windows, don't worry this is a known Huggingface error-
# Please use the code below which wille import the data we were planning to use from a .txt file

#origin of this data : data/FinancialPhraseBanl-v1.0

df = pd.read_csv('data\\FinancialPhraseBank-v1.0\\Sentences_50Agree.txt',
            encoding = 'ISO-8859-1',on_bad_lines='skip',sep = '.@')
df.columns = ['sentence','label']
df['label'] = df['label'].replace(to_replace=({'neutral':0,'positive':1,'negative':2}))
df['label']= df['label'].astype(int)

In [11]:
### Tokenizer 

In [12]:
df.label.value_counts(normalize = True)

0    0.594014
1    0.281321
2    0.124665
Name: label, dtype: float64

In [13]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [14]:
#how does the tokenizer work ? 
print('\nThis is our input sentence : \n Hi my name is BERT and I am overjoyed  to meet you ! \n')

out = tokenizer(['Hi my name is BERT and I am overjoyed  to meet you ! '],
          max_length=64,padding="max_length", truncation=True,return_tensors='pt')
print('These are the outputs of the tokenizer:\n')
print(out)

print('\nThese inputs correspond to the original sentence with separation and padding thrown in :\n')
print([tokenizer.decode(i) for i in out['input_ids']])


This is our input sentence : 
 Hi my name is BERT and I am overjoyed  to meet you ! 

These are the outputs of the tokenizer:

{'input_ids': tensor([[  101,  8790,  1139,  1271,  1110,   139,  9637,  1942,  1105,   146,
          1821,  1166, 18734,  1174,  1106,  2283,  1128,   106,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
     

In [15]:
# Now that we covered  the tokenizer lets introduce the other building block : the model 

print('this is our model : \n')
model = AutoModelForSequenceClassification.from_pretrained('bert-base-cased')
layers = [i for i in model.parameters()]
print('\n First layer shape (vocabulary size) : \n ',layers[0].shape,
      '\n Number of self attention heads: ',len([i.shape  for i in layers if (i.shape==torch.Size([3072])) ]),
'\n Last layer shape (prediction task output shape) : \n ',layers[-1].shape)

this is our model : 


 First layer shape (vocabulary size) : 
  torch.Size([28996, 768]) 
 Number of self attention heads:  12 
 Last layer shape (prediction task output shape) : 
  torch.Size([2])


In [16]:
# basic forward propagation of our BERT model 
print('This is our forward propagation syntax. \n We feed in a tokenized text and receive the \n predicted  logits over the 2 classes : \n')
model_output = model.forward(**out)
print(model_output.logits)

This is our forward propagation syntax. 
 We feed in a tokenized text and receive the 
 predicted  logits over the 2 classes : 

tensor([[0.4713, 0.1332]], grad_fn=<AddmmBackward0>)


In [17]:
# Working with BERT hands-on 

In [18]:
#  define tokenizer & model 
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

# turn the configuration for a 3 sentiment classification task
config = AutoConfig.from_pretrained('bert-base-cased')
config.num_labels = 3

model = AutoModelForSequenceClassification.from_config(config)


In [19]:
train, test = train_test_split(df, test_size=0.25, random_state=96)
test, val = train_test_split(test, test_size=0.4, random_state=96)

In [20]:
print(train.label.value_counts(normalize=True),'\n',
test.label.value_counts(normalize=True),'\n',
val.label.value_counts(normalize=True))

0    0.593174
1    0.282136
2    0.124690
Name: label, dtype: float64 
 0    0.583219
1    0.294360
2    0.122421
Name: label, dtype: float64 
 0    0.616495
1    0.255670
2    0.127835
Name: label, dtype: float64


## Defining a dataset class to interact with the Huggingface Transformers

In [21]:
# Defining a Dataset object to put our data in


class BERTTutorialDataset(Dataset):
    """
    Special dataset class built on top of the torch Dataset class
    useful to have memory efficient dataloading tokenization batching and trainning.
    
    Huggingface can use these types of dataset as inputs and run all trainning/prediction on them. 
    """
    def __init__(self, input_data, sentiment_targets, tokenizer, max_len):
        """
        Basic generator function for the class.
        -----------------
        input_data : array
            Numpy array of string  input text to use for downstream task 
        sentiment_targets : 
            Numpy array of integers indexed in  the pytorch style of [0,C-1] with C being the total number of classes
            In our example this means the target sentiments should range from 0 to 2. 
        tokenizer  : Huggingface tokenizer 
            The huggingface tokenizer to use
        max_len : 
            The truncation length of the tokenizer 
        -------------------
        
        Returns : 
        
            Tokenized text with inputs, attentions and labels, ready for the Training script. 
        """
        self.input_data = input_data
        self.sentiment_targets = sentiment_targets
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __len__(self):
        """
        Function required by torch huggingface to batch efficiently
        """
        return len(self.input_data)
    
    def __getitem__(self, item):
        text = str(self.input_data[item])
        target = self.sentiment_targets[item]
        # only difference with the previuous tokenization step is the encode-plus for special tokens
        encoding = self.tokenizer.encode_plus(
          text,
          add_special_tokens=True,
          max_length=self.max_len,
          return_token_type_ids=False,
          padding='max_length',
          return_attention_mask=True,
          return_tensors='pt',
          truncation = True
        )
        return {
          'text': text,
          'input_ids': encoding['input_ids'].flatten(),
          'attention_mask': encoding['attention_mask'].flatten(),
          'labels': torch.tensor(target, dtype=torch.long)
        }

In [22]:
# Creating our train-val-test datasets
MAX_LEN = 128
train_ds = BERTTutorialDataset(
    input_data=train['sentence'].to_numpy(),
        sentiment_targets=train['label'].to_numpy(),
        tokenizer=tokenizer,
        max_len=MAX_LEN
    )
val_ds = BERTTutorialDataset(
    input_data=val['sentence'].to_numpy(),
        sentiment_targets=val['label'].to_numpy(),
        tokenizer=tokenizer,
        max_len=MAX_LEN
    )

test_ds = BERTTutorialDataset(
    input_data=test['sentence'].to_numpy(),
        sentiment_targets=test['label'].to_numpy(),
        tokenizer=tokenizer,
        max_len=MAX_LEN
    )


In [23]:
# Define some accuracy measure ( helpful for the early stopping )
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import numpy as np
def compute_metrics(p):
    """
    Function to calculate accuracies and losses for the validation from the predicted outputs
    This is neccessary for the early stopping. 
    """
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average='macro')
    precision = precision_score(y_true=labels, y_pred=pred, average='macro')
    f1 = f1_score(y_true=labels, y_pred=pred, average='macro')    
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}


## Defining the trainning arguments

In [24]:
# Define trainning arguments 
training_args = TrainingArguments('BERT_TUTORIAL_MODEL', overwrite_output_dir=True, evaluation_strategy="steps", 
                                  num_train_epochs=50, weight_decay=1e-8,learning_rate=1e-5,
                                  eval_steps=50,metric_for_best_model='accuracy',
                                 per_device_train_batch_size=16, per_device_eval_batch_size=16,
                                 load_best_model_at_end = True, save_total_limit=10, save_steps=50,no_cuda=False,
                             fp16=True,gradient_accumulation_steps=4)
trainer = Trainer(
    model =model, args=training_args, train_dataset=train_ds, eval_dataset=val_ds,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=5)], compute_metrics=compute_metrics
)

Using amp half precision backend


### Lauching the training preocess 

In [25]:
# If you want to run the trainer run the code below ( it takes about 12 minutes )
# Its as easy as runnning trainer.train()

#trainer.train()

In [26]:
# If you want to evaluate the trainer run the code below

#trainer.evaluate(test_ds)
#predictions = trainer.predict(test_ds)
#output = np.argmax(predictions.predictions,1)
#sns.heatmap(confusion_matrix(test.label.values,output),annot =confusion_matrix(test.label.values,output) )#,labels = ['1','-1','0']

#del predictions --> We delete the predictions as we don't want to occupy too much gpu space

In [27]:
# deleting all our objects to save GPU space
del model
del trainer
del train_ds
del val_ds
del test_ds
del tokenizer
torch.cuda.empty_cache()

## Defining the trainning arguments

In [28]:
# Try with finbert 
#  define tokenizer & model --> this is just a change from the previous code above
finbert_tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

# turn the configuration for a 3 sentiment classification task
config = AutoConfig.from_pretrained("ProsusAI/finbert")
config.num_labels = 3

finbert_model = AutoModelForSequenceClassification.from_config(config)

# redefine our datasets as wechanged the tokenizer 

MAX_LEN = 128
train_ds = BERTTutorialDataset(
    input_data=train['sentence'].to_numpy(),
        sentiment_targets=train['label'].to_numpy(),
        tokenizer=finbert_tokenizer,
        max_len=MAX_LEN
    )
val_ds = BERTTutorialDataset(
    input_data=val['sentence'].to_numpy(),
        sentiment_targets=val['label'].to_numpy(),
        tokenizer=finbert_tokenizer,
        max_len=MAX_LEN
    )

test_ds = BERTTutorialDataset(
    input_data=test['sentence'].to_numpy(),
        sentiment_targets=test['label'].to_numpy(),
        tokenizer=finbert_tokenizer,
        max_len=MAX_LEN
    )


loading configuration file https://huggingface.co/ProsusAI/finbert/resolve/main/config.json from cache at C:\Users\drago/.cache\huggingface\transformers\2120f4f96b5830e5a91fe94d242471b0133b0976c8d6e081594ab837ac5f17bc.ef97278c578016c8bb785f15296476b12eae86423097fed78719d1c8197a3430
Model config BertConfig {
  "_name_or_path": "ProsusAI/finbert",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "positive",
    "1": "negative",
    "2": "neutral"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "negative": 1,
    "neutral": 2,
    "positive": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "ab

### Lauching the training preocess 

In [29]:
# Define trainning arguments 
training_args = TrainingArguments('FINBERT_TUTORIAL_MODEL', overwrite_output_dir=True, evaluation_strategy="steps", 
                                  num_train_epochs=50, weight_decay=1e-8,learning_rate=1e-5,
                                  eval_steps=50,metric_for_best_model='accuracy',
                                 per_device_train_batch_size=16, per_device_eval_batch_size=16,
                                 load_best_model_at_end = True, save_total_limit=10, save_steps=50,no_cuda=False,
                             fp16=True,gradient_accumulation_steps=4)
trainer = Trainer(
    model =finbert_model, args=training_args, train_dataset=train_ds, eval_dataset=val_ds,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=5)], compute_metrics=compute_metrics
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using amp half precision backend


In [30]:
# If you want to run the trainer run the code below ( it takes about 10 minutes )

#trainer.train()

In [31]:
# If you want to evaluate the trainer run the code below

#trainer.evaluate(test_ds)
#predictions = trainer.predict(test_ds)
#output = np.argmax(predictions.predictions,1)
#sns.heatmap(confusion_matrix(test.label.values,output),annot =confusion_matrix(test.label.values,output))

#del predictions

In [32]:
# deleting all our objects to save GPU space
del finbert_model
del trainer
del train_ds
del val_ds
del test_ds
del finbert_tokenizer
torch.cuda.empty_cache()

# Masked Language Modelling code

In [33]:
# We will not be running the code below as it would take a long time

```python
# Import the model & tokenizer

model_to_pretrain = BertForMaskedLM.from_pretrained('bert-base-uncased')
tokenizer_for_pretraining = AutoTokenizer.from_pretrained("bert-base-cased")

# tokenize the inputs of the text 
inputs_for_pretraining = tokenizer_for_pretraining(df.sentence.tolist(), return_tensors='pt', max_length=32, truncation=True, padding='max_length')
inputs_for_pretraining['labels'] = inputs.input_ids.detach().clone()

# create random array of floats with equal dimensions to input_ids tensor
random_mask = torch.rand(inputs_for_pretraining.input_ids.shape)

# create mask array --> we hide 15% of the inputs for the masked language modelling task  
mask_arr = (random_mask < 0.15) * (inputs_for_pretraining.input_ids != 101) * \
           (inputs_for_pretraining.input_ids != 102) * (inputs_for_pretraining.input_ids != 0)

selection = []

for i in range(inputs_for_pretraining.input_ids.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
    )
for i in range(inputs_for_pretraining.input_ids.shape[0]):
    inputs_for_pretraining.input_ids[i, selection[i]] = 103
    
    
class TUTORIALDataset(torch.utils.data.Dataset):
    """
    This is also a Dataset class as the Dataset class before 
    """
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)
    
    
dataset = TUTORIALDataset(inputs_for_pretraining)
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)


device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# Move model to device
model_to_pretrain.to(device)
# launch model training
model_to_pretrain.train()

args = TrainingArguments(
    output_dir='out',
    per_device_train_batch_size=16,
    num_train_epochs=2 #only 2 train epochs --> this is a toy example, running a proper script would take upwards of 10 hours ! 
)

MLM_trainer = Trainer(
    model=model_to_pretrain,
    args=args,
    train_dataset=dataset
)

MLM_trainer.train()
```

# 4. Visualisation

In [34]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers_interpret import SequenceClassificationExplainer

To visualise which words in each phrase are the most important for the prediction we will use the python package transformers_interpret 

In [35]:
fin_model_name = "ProsusAI/finbert"
model_name = "textattack/bert-base-uncased-SST-2"


fin_model = AutoModelForSequenceClassification.from_pretrained(fin_model_name)
fin_tokenizer = AutoTokenizer.from_pretrained(fin_model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

loading configuration file https://huggingface.co/ProsusAI/finbert/resolve/main/config.json from cache at C:\Users\drago/.cache\huggingface\transformers\2120f4f96b5830e5a91fe94d242471b0133b0976c8d6e081594ab837ac5f17bc.ef97278c578016c8bb785f15296476b12eae86423097fed78719d1c8197a3430
Model config BertConfig {
  "_name_or_path": "ProsusAI/finbert",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "positive",
    "1": "negative",
    "2": "neutral"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "negative": 1,
    "neutral": 2,
    "positive": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "ab

In [36]:
# With both the model and tokenizer initialized we are now able to get explanations on an example text.
cls_explainer = SequenceClassificationExplainer(model,
                                                tokenizer)

fin_cls_explainer = SequenceClassificationExplainer(fin_model,
                                                    fin_tokenizer)

In [37]:
word_attributions = cls_explainer("Pharmaceuticals group Orion Corp reported a fall in its third-quarter earnings that were hit by larger expenditures on R&D and marketing")
word_attributions = fin_cls_explainer("Pharmaceuticals group Orion Corp reported a fall in its third-quarter earnings that were hit by larger expenditures on R&D and marketing")

In [38]:
cls_explainer.predicted_class_name

'LABEL_0'

In [39]:
bert_vis = cls_explainer.visualize()

In [40]:
fin_bert_vis = fin_cls_explainer.visualize()

| Model name        | Loughram McDonald | Naive Bayes | Bert   | FinBert |
|-------------------|-------------------|-------------|--------|---------|
| Averaged F1 Score | 0.2544            | 0.3954      | 0.7564 |         |